**Copyright 2021 The TensorFlow Hub Authors.**

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/senteval_for_universal_sentence_encoder_cmlm.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/senteval_for_universal_sentence_encoder_cmlm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/senteval_for_universal_sentence_encoder_cmlm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>
</td>
  <td><a href="https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png"> TF Hub モデルを参照</a></td>
</table>

# ユニバーサルセンテンスエンコーダ SentEval の実演

この Colab は、文の埋め込みの品質を測定するためのライブラリである [SentEval](https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1) ツールキットを使用した[ユニバーサルセンテンスエンコーダ CMLM モデル](https://github.com/facebookresearch/SentEval)を説明します。 SentEval ツールキットには、埋め込みモデルの一般化能力を評価し、エンコードされた言語プロパティを評価できる、さまざまな下流タスクのセットが含まれています。

最初の 2 つのコードブロックを実行して環境をセットアップします。3 番目のコードブロックでは、SentEval タスクを選択してモデルを評価できます。この Colab を実行するには、GPU ランタイムをお勧めします。

ユニバーサルセンテンスエンコーダ CMLM モデルの詳細については、https://openreview.net/forum?id=WDVD4lUCTzU を参照してください。

In [ ]:
#@title Install dependencies
!pip install --quiet "tensorflow-text==2.8.*"
!pip install --quiet torch==1.8.1

## SentEvalとタスクデータをダウンロードする

このステップでは、github から SentEval をダウンロードし、データスクリプトを実行してタスクデータをダウンロードします。完了するまでに最大 5 分かかる場合があります。

In [ ]:
#@title Install SentEval and download task data
!rm -rf ./SentEval
!git clone https://github.com/facebookresearch/SentEval.git
!cd $PWD/SentEval/data/downstream && bash get_transfer_data.bash > /dev/null 2>&1

＃SentEval 評価タスクの実行。次のコードブロックは SentEval タスクを実行して結果を出力し、次のタスクのいずれかを選択して USE CMLM モデルを評価します。

```
MR	CR	SUBJ	MPQA	SST	TREC	MRPC	SICK-E
```

実行するモデル、パラメーター、およびタスクを選択します。ラピッドプロトタイピングパラメータは、計算時間を短縮するために使用でき、より速く結果を得ることができます。

通常、**'rapid prototyping'** パラメータを使用してタスクを完了するには 5〜15 分かかり、**'slower, best performance'** パラメータでは最大 1 時間かかります。

```
params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}
```

より良い結果を得るには、より遅い **'slower, best performance'** パラメータを使用してください。計算には最大 1 時間かかる場合があります。

```
params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 6}
```


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(f'{os.getcwd()}/SentEval')

import tensorflow as tf

# Prevent TF from claiming all GPU memory so there is some left for pytorch.
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Memory growth needs to be the same across GPUs.
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

import tensorflow_hub as hub
import tensorflow_text
import senteval
import time

PATH_TO_DATA = f'{os.getcwd()}/SentEval/data'
MODEL = 'https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1' #@param ['https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1', 'https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-large/1']
PARAMS = 'rapid prototyping' #@param ['slower, best performance', 'rapid prototyping']
TASK = 'CR' #@param ['CR','MR', 'MPQA', 'MRPC', 'SICKEntailment', 'SNLI', 'SST2', 'SUBJ', 'TREC']

params_prototyping = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_prototyping['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}

params_best = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params_best['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 6}

params = params_best if PARAMS == 'slower, best performance' else params_prototyping

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1")

inputs = tf.keras.Input(shape=tf.shape(''), dtype=tf.string)
outputs = encoder(preprocessor(inputs))

model = tf.keras.Model(inputs=inputs, outputs=outputs)

def prepare(params, samples):
    return

def batcher(_, batch):
    batch = [' '.join(sent) if sent else '.' for sent in batch]
    return model.predict(tf.constant(batch))["default"]


se = senteval.engine.SE(params, batcher, prepare)
print("Evaluating task %s with %s parameters" % (TASK, PARAMS))
start = time.time()
results = se.eval(TASK)
end = time.time()
print('Time took on task %s : %.1f. seconds' % (TASK, end - start))
print(results)


詳細情報

- その他のテキスト埋め込みモデルは、[TensorFlow Hub](https://tfhub.dev) をご覧ください。
- [多言語ユニバーサルセンテンスエンコーダ CMLM モデル](https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1)もご覧ください。
- 他の[ユニバーサルセンテンスエンコーダモデル](https://tfhub.dev/google/collections/universal-sentence-encoder/1)を確認してください

## リファレンス

- Ziyi Yang、Yinfei Yang、Daniel Cer、Jax Law、Eric Darve。[Universal Sentence Representations Learning with Conditional Masked Language Model. November 2020](https://openreview.net/forum?id=WDVD4lUCTzU)
